In [1]:
!pip install quickle

   ---------------------------------------- 0.0/46.5 kB ? eta -:--:--
   -------- ------------------------------- 10.2/46.5 kB ? eta -:--:--
   -------------------------- ------------- 30.7/46.5 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 46.5/46.5 kB 465.3 kB/s eta 0:00:00


In [4]:
!curl -O https://raw.githubusercontent.com/NathanDonagi/NYT-Game-Solvers/main/answers.txt
!curl -O https://raw.githubusercontent.com/NathanDonagi/NYT-Game-Solvers/main/wordlist.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12929  100 12929    0     0  93362      0 --:--:-- --:--:-- --:--:-- 95066
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 89129  100 89129    0     0   635k      0 --:--:-- --:--:-- --:--:--  644k


In [5]:
from collections import defaultdict
import quickle
import os

with open('wordlist.txt', 'r') as f:
    word_list = f.read().splitlines()

with open('answers.txt', 'r') as f:
    answer_list = f.read().splitlines()

pairings = defaultdict(lambda: {})

def get_result(answer, guess):
    if answer in pairings and guess in pairings[answer]:
        return pairings[answer][guess]
    out = ["b"] * 5
    a_count = defaultdict(lambda: 0)
    g_count = defaultdict(lambda: 0)
    for a, b in zip(answer, guess):
        a_count[a] += 1
        g_count[b] += 1

    for i, (a, b) in enumerate(zip(answer, guess)):
        if a == b:
            out[i] = 'g'
            a_count[a] -= 1
            g_count[b] -= 1

    for i, b in enumerate(guess):
        if out[i] == 'g':
            continue
        if a_count[b]:
            a_count[b] -= 1
            out[i] = 'y'
    return "".join(out)

ModuleNotFoundError: No module named 'quickle'

In [3]:
if os.path.exists('pairings.pickle'):
    print("precompute exists, begun reading")
    with open('pairings.pickle', 'rb') as f:
        pairings = quickle.loads(f.read())
    print("precompute loaded")
else:
    for answer in word_list:
        print(answer)
        for guess in word_list:
            res = get_result(answer, guess)
            pairings[answer][guess] = res
            if not res in pairings[guess]:
                pairings[guess][res] = []
            pairings[guess][res].append(answer)
    with open('pairings.pickle', 'wb') as f:
        f.write(quickle.dumps(dict(pairings)))
    print("precompute finished")


precompute exists, begun reading
precompute loaded


In [4]:
mem1 = {}
mem2 = {}

def score_guess(guess, prev_guesses, possible_guesses, possible_answers, mn): 
    s = 0
    for answer in possible_answers:
        if s>=mn:
            return s

        res = get_result(answer, guess)
        if res == 'ggggg':
           continue
    
        if (possible_guesses, possible_answers, guess, res) not in mem2:
            mem2[(possible_guesses, possible_answers, guess, res)] = (tuple(filter(lambda x: x in pairings[guess][res], possible_guesses)), tuple(filter(lambda x: x in pairings[guess][res], possible_answers)))
        
        possible_next_guesses, possible_next_answers = mem2[(possible_guesses, possible_answers, guess, res)]
        if len(possible_next_guesses) == len(possible_guesses):
            return float('inf')
        s += score(prev_guesses, possible_next_guesses, possible_next_answers)[0]
    return s
    
def score(prev_guesses, possible_guesses, possible_answers):
    if (possible_guesses, possible_answers) in mem1:
        return mem1[(possible_guesses, possible_answers)]
    if len(possible_answers) == 1:
        mem1[(possible_guesses, possible_answers)] = 1, possible_answers[0]
        return mem1[(possible_guesses, possible_answers)]
    if len(prev_guesses) > 7:
        return float('inf'), None

    mn, best = float('inf'), None
    
    for guess in possible_answers+tuple(filter(lambda x: x in possible_answers, possible_guesses)):
        temp = score_guess(guess, prev_guesses+(guess,), possible_guesses, possible_answers, mn)
        if temp < mn:
            mn = temp
            best = guess
        if mn == 0:
            break

    mem1[(possible_guesses, possible_answers)] = 1+mn/len(possible_answers), best
    return  mem1[(possible_guesses, possible_answers)]

In [5]:
def get_next_guess(guesses):
    possible_guesses = word_list
    possible_answers = answer_list
    for guess, res in guesses:
        if res not in pairings[guess]:
            pairings[guess][res] = []
        possible_guesses = tuple(filter(lambda x: x in pairings[guess][res], possible_guesses))
        possible_answers = tuple(filter(lambda x: x in pairings[guess][res], possible_answers)) 
        if res == 'ggggg':
            return 0, None
    print(len(possible_guesses), len(possible_answers))
    out = score(('salet',), possible_guesses, possible_answers)
    # print(out, possible_guesses, possible_answers)
    return out

def solve(answer):
    if answer == 'salet':
        return 1
    guesses = ['salet']
    # print('first')
    temp = get_next_guess([(guess, get_result(answer, guess)) for guess in guesses])[1]
    while temp != answer:
        temp = get_next_guess([(guess, get_result(answer, guess)) for guess in guesses])[1]
        print(temp)
        if temp in guesses:
            raise NotImplementedError
        guesses.append(temp)
    # print(guesses)
    return len(guesses)

In [ ]:
#generate all possible solves
s=0
for i, answer in enumerate(answer_list):
    print(answer)
    s += solve(answer)
    print(s/(i+1))

#average number of guesses taken
print(s/len(answer_list))

In [12]:
get_next_guess([('salet', 'bbybb'), ('cloud', 'bgbbb'), ('bling', 'gggbb')])[1]

2 1


'blimp'

In [8]:
#load saved memory
if os.path.exists('mem1.pickle'):
    print("mem1 precompute exists, begun reading")
    with open('mem1.pickle', 'rb') as f:
        mem1 = quickle.loads(f.read())
    print("precompute loaded")

if os.path.exists('mem2.pickle'):
    print("mem2 precompute exists, begun reading")
    with open('mem2.pickle', 'rb') as f:
        mem2 = quickle.loads(f.read())
    print("precompute2 loaded")

mem1 precompute exists, begun reading
precompute loaded
mem2 precompute exists, begun reading
precompute2 loaded


In [15]:
#save memory
with open('mem1.pickle', 'wb') as f:
    f.write(quickle.dumps(dict(mem1)))
with open('mem2.pickle', 'wb') as f:
    f.write(quickle.dumps(dict(mem2)))